In [1]:
import os
import math
import tflearn
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
import sklearn.linear_model as model
import csv
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import sklearn
from sklearn.naive_bayes import BernoulliNB
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
import nltk
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI

Using TensorFlow backend.


### Basic Neural Network

Use Michelle's method for importing data since I can't do it correctly:

In [2]:
train_data = np.genfromtxt('Data/training_data.txt',dtype='str')
test_data = np.genfromtxt('Data/test_data.txt',dtype='str')
#train_data = np.loadtxt('training_data.txt',  delimiter=' ')
#test_data = np.loadtxt('test_data.txt')
train_labels = train_data[0, :]
train_stars = train_data[1:, 0]
train_reviews = train_data[1:, 1:]

test_labels = test_data[0, :]
#test_stars = train_data[1:, 0]
test_reviews = test_data[1:, 0:]

trainX = train_reviews
trainY = train_stars
testX = test_reviews
#testY = test_stars

print("trainX = ", trainX.shape)
print("trainY = ", trainY.shape)
print("testX = ", testX.shape)


for i in range(0, len(trainX)):
    for j in range(0, len(trainX[0])):
        trainX[i][j] = int(trainX[i][j])

for i in range(0, len(testX)):
    for j in range(0, len(testX[0])):
        testX[i][j] = int(testX[i][j])

for i in range(0, len(trainY)):
        trainY[i] = int(trainY[i])
#for i in range(0, len(trainY)):
        #testY[i] = int(testY[i])

trainX =  (20000, 1000)
trainY =  (20000,)
testX =  (10000, 1000)


In [3]:
print(train_labels)

['Label' 'thi' 'book' ..., 'pair' 'soni' 'ten']


Define a function for classification error:

In [4]:
def class_error(y, y_pred):
    misclassified = 0
    for i in range(len(y)):
        if y[i] != y_pred[i]:
            misclassified += 1
    return float(misclassified) / len(y)

Split data into 80% training set and 20% test set.

In [5]:
num_rows = []
for i in range(len(trainY)):
    num_rows.append(i)
    
np.random.shuffle(num_rows)
train_indices = num_rows[:16000]
valid_indices = num_rows[16000:]

train_X = []
train_y = []
valid_X = []
valid_y = []

for index in train_indices:
    train_X.append(trainX[index])
    train_y.append(trainY[index])
    
for index in valid_indices:
    valid_X.append(trainX[index])
    valid_y.append(trainY[index])

With Michelle's code, it doesn't actually convert the array contents into integers because she tried to do it in place. This code does it correctly out of place:

In [6]:
train_X2 = []
for row in train_X:
    temp = []
    for element in row:
        temp.append(int(element))
    train_X2.append(temp)

valid_X2 = []
for row in valid_X:
    temp = []
    for element in row:
        temp.append(int(element))
    valid_X2.append(temp)
    
valid_y2 = []
for row in valid_y:
    valid_y2.append(int(row))
    
train_y2 = []
for row in train_y:
    train_y2.append(int(row))
    
test_X2 = []
for row in testX:
    temp = []
    for element in row:
        temp.append(int(element))
    test_X2.append(temp)

Convert everything back to a numpy array:

In [7]:
train_X2 = np.array(train_X2)
valid_X2 = np.array(valid_X2)
train_y2 = np.array(train_y2)
valid_y2 = np.array(valid_y2)
test_X2 = np.array(test_X2)

# Naive Bayes Algorithm

In [8]:
train_y2 = to_categorical(train_y2, 2)

In [9]:
print(train_y2)

[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]


In [10]:
valid_y2 = to_categorical(valid_y2, 2)
print(valid_y2)

[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 ..., 
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]


In [11]:
trainY_1D = []
for i in range(0, len(train_y2)):
    if train_y2[i][0]==1:
        trainY_1D.append(0)
    else:
        trainY_1D.append(1)



In [12]:
validY_1D = []
for i in range(0, len(valid_y2)):
    if valid_y2[i][0]==1:
        validY_1D.append(0)
    else:
        validY_1D.append(1)

In [13]:
print(len(trainY))
print(len(all_trainX))

20000


NameError: name 'all_trainX' is not defined

In [14]:
all_trainY = []
for i in range(0, len(trainY)):
    if trainY[i][0]==1:
        all_trainY.append(0)
    else:
        all_trainY.append(1)


In [15]:
all_trainX = []
for row in trainX:
    temp = []
    for element in row:
        temp.append(int(element))
    all_trainX.append(temp)

In [16]:
print(len(all_trainX))
print(len(train_labels))

20000
1001


# Naive Bayes

In [17]:
def TFIDF(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element)):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    tokenize = lambda doc: doc.lower().split(" ")
    sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    return sklearn_tfidf.fit_transform(documents)

In [18]:
def bigramReturner (tweetString):
    tweetString = tweetString.lower()
    bigramFeatureVector = []
    for item in nltk.bigrams(tweetString.split()):
        bigramFeatureVector.append(' '.join(item))
    return bigramFeatureVector

In [19]:
def extract_features(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element)):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features


In [27]:
nb = MultinomialNB(alpha=0.1).fit(TFIDF(train_X2, train_labels), trainY_1D)
nb.get_params()
class_error(nb.predict(TFIDF(valid_X2, train_labels)), validY_1D)
prediction = nb.predict(TFIDF(test_X2, train_labels))
writeToText(prediction, 'NaiveBayes.txt')

In [29]:
vectorizer = CountVectorizer()
nb = MultinomialNB(alpha=0.1).fit(HashingVectorizer(norm=None, non_negative=True).fit_transform(test_data), trainY_1D)

class_error(nb.predict(TFIDF(valid_X2, train_labels)), validY_1D)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [ ]:
nb = (BernoulliNB(alpha=.01))
nb = MultinomialNB(alpha=0.1).fit(HashingVectorizer(norm=None, non_negative=True).fit_transform(test_data), trainY_1D)

nb.get_params()
class_error(nb.predict(TFIDF(valid_X2, train_labels)), validY_1D)


In [ ]:
documents = []

for element in train_X2:
    temp = ''
    for i in range(len(element)):
        word = train_labels[i + 1] + ' '
        new_word = word * element[i]
        temp += (new_word)
    documents.append(temp)

print(documents)
#for item in nltk.bigrams (text.split()): print ' '.join(item)

In [ ]:
bigrameturner(documents)

# Gaussian Naive Bayes

In [ ]:
gnb = GaussianNB()
#gnb = gnb.fit(train_X2, trainY_1D)
print(len(all_trainX))
print(len(all_trainY))
gnb = gnb.fit(all_trainX, all_trainY)

In [ ]:
mnb = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)
#gnb = gnb.fit(train_X2, trainY_1D)
print(len(all_trainX))
print(len(all_trainY))
mnb = mnb.fit(all_trainX, all_trainY)

In [ ]:
bnb = BernoulliNB(alpha=50.0, class_prior=None, fit_prior=False)
#gnb = gnb.fit(train_X2, trainY_1D)
print(len(all_trainX))
print(len(all_trainY))
bnb = bnb.fit(all_trainX, all_trainY)

Fit the model:

Print the classification error on the training set:

In [ ]:
# Check error for Gaussian
prediction = gnb.predict(train_X2)
training_err = class_error(trainY_1D, prediction)
print("training_err", training_err)

prediction_V = gnb.predict(test_X2)
v_err = class_error(validY_1D, prediction_V)
print("validation error", v_err)

In [ ]:
# Check error for Bernoulli
prediction = bnb.predict(train_X2)
training_err = class_error(trainY_1D, prediction)
print("training_err", training_err)

prediction_V = bnb.predict(test_X2)
v_err = class_error(validY_1D, prediction_V)
print("validation error", v_err)


In [ ]:
print(train_X2)

In [ ]:
prediction = mnb.predict(train_X2[0:30])
print(prediction)

In [ ]:
def TFIDF(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element)):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    tokenize = lambda doc: doc.lower().split(" ")
    sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    return sklearn_tfidf.fit_transform(documents)

In [ ]:
# Check error for Multinomial
prediction = mnb.predict(train_X2)
training_err = class_error(trainY_1D, prediction)
print("training_err", training_err)

prediction_V = mnb.predict(test_X2)
v_err = class_error(validY_1D, prediction_V)
print("validation error", v_err)

In [ ]:
TFIDF(train_X2, train_labels)

In [ ]:
mnb = MultinomialNB(alpha=50).fit(TFIDF(train_X2, train_y2), train_y2)
#mnb.fit(train_X2, train_y2)
class_error(mnb.predict(valid_X2), valid_y2)

In [ ]:

testXTF = TFIDF(test_X2, test_labels)
# print(np.shape(test_X2), np.shape(test_labels), np.shape(train_labels), np.shape(train_X2))
# print(np.shape(testXTF))

In [ ]:
test_predictions = model.predict(test_X2)
# test_predictions = model.predict(testXTF)
predictions_arr = []

for i in range(0, len(test_predictions)):
    if abs(test_predictions[i][0]-1) < abs(test_predictions[i][1]-1):
        predictions_arr.append(0)
    else:
        predictions_arr.append(1)
    
writeToText(predictions_arr, 'basicNeuralNetNOTF.txt')

Print the classification error on the validation set:

A function for writing to the text file for the submission:

In [26]:
def writeToText(predictions, filename):
    array = [["Id","Prediction"]]
    index = 1
    for i in range(len(predictions)):
        predict = predictions[index - 1]
        array.append([index, int(predictions[index - 1])])
        index += 1
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter=',', quotechar='|')
    writer.writerows(array)

### Logistic Regression with TF-IDF

A function to convert the set into a TF-IDF matrix:

In [25]:
def TFIDF(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element) - 1):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    tokenize = lambda doc: doc.lower().split(" ")
    sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    return sklearn_tfidf.fit_transform(documents)

Fit the model using the training data:

In [ ]:
model = Sequential()
model.add(Dense(1000, input_dim=1000))
model.add(Activation('relu'))
model.add(Dropout(0.20))

model.add(Dense(1000)) 
model.add(Activation('relu'))
model.add(Dropout(0.20))

model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

inputX = TFIDF(train_X2, train_labels)
print(inputX)
print(len(inputX))

fit = model.fit(inputX, trainY, batch_size=64, nb_epoch=15, validation_data=(validX, validY),
    verbose=1)


prediction = model.predict(trainX)
predictedY = []

trainY_new = []
for i in range(0, len(trainY)):
    if trainY[i][0] == 1:
        trainY_new.append(0)
    else:
        trainY_new.append(1)



for i in range(0, len(prediction)):
    if abs(prediction[i][0]-1) < abs(prediction[i][1]-1):
        predictedY.append(0)
    else:
        predictedY.append(1)
        
class_error(predictedY, trainY_new)